In [209]:
# using binomial model to approximate balck scholes model
import math
import scipy.special
import numpy as np
def american_call_option_price(T,S_0,r,c,sigma,n,K): # it's never optimal to exercise an american call opiton before the expiration
    dt = T/n
    R_n = math.exp(r*dt)
    R_C = math.exp((r-c)*dt)
    u_n = math.exp(sigma*(dt**0.5))
    d_n = 1/u_n
    q_n = (R_C-d_n)/(u_n-d_n)
    # value sequence
    s_n = [S_0*u_n**(n-i)*d_n**i for i in range(n+1)]
    binom_coef = np.array([scipy.special.binom(n,i)*q_n**(n-i)*(1-q_n)**(i) for i in range(n+1)])
    payoff = np.array([max(x-K,0) for x in s_n])
    return sum(binom_coef*payoff)/(R_n**n)  #return the discounted value

In [210]:
q1=american_call_option_price(0.25,100,0.02,0.01,0.3,15,110)

In [211]:
q1

2.604077132966558

In [212]:
#american put option
#actually this is a stock price tree
def american_put_option_tree(T,S_0,r,c,sigma,n): 
    dt = T/n
    R_n = math.exp(r*dt)
    R_C = math.exp((r-c)*dt)
    u_n = math.exp(sigma*(dt**0.5))
    d_n = 1/u_n
    q_n = (R_C-d_n)/(u_n-d_n)
    tree = np.zeros((n+1,n+1))
    for i in range(n+1):
        for j in range(i+1):
            tree[i][j]=S_0*u_n**j*d_n**(i-j)
    return tree

In [213]:
american_put_option_tree(0.25,100,0.02,0.01,0.3,15)

array([[100.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ],
       [ 96.20105771, 103.94896104,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ],
       [ 92.54643505, 100.        , 108.05386501,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ],
       [ 89.03064939,  96.20105771, 103.94896104, 112.32087004,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,  

In [244]:
# american put option price tree
def american_put_option_price_and_early_stopping(T,S_0,r,c,sigma,n,K):
    dt = T/n
    R_n = math.exp(r*dt)
    R_C = math.exp((r-c)*dt)
    u_n = math.exp(sigma*(dt**0.5))
    d_n = 1/u_n
    q_n = (R_C-d_n)/(u_n-d_n)
    stopping_time=n
    price_tree = american_put_option_tree(T,S_0,r,c,sigma,n)
    f = np.vectorize(lambda x:max(x,0.0))
    payoff=f(K-price_tree)
    for i in range(n-1,-1,-1):
        for j in range(i+1):
            price = 1/R_n*(q_n*payoff[i+1][j+1]+(1-q_n)*payoff[i+1][j])
            if payoff[i][j]> price:
                stopping_time = i
            else:
                payoff[i][j]= price
    return payoff[0][0],stopping_time
    

In [245]:
american_put_option_price_and_early_stopping(0.25,100,0.02,0.01,0.3,15,110)

(12.359784797284899, 5)

In [246]:
def future_price_lattice(T,S_0,r,c,sigma,n):
    dt = T/n
    R_n = math.exp(r*dt)
    R_C = math.exp((r-c)*dt)
    u_n = math.exp(sigma*(dt**0.5))
    d_n = 1/u_n
    q_n = (R_C-d_n)/(u_n-d_n)
    price_tree = american_put_option_tree(T,S_0,r,c,sigma,n)
    future_price_tree = np.zeros((n+1,n+1))
    future_price_tree[n]=price_tree[n]
    for i in range(n-1,-1,-1):
        for j in range(i+1):
            future_price_tree[i][j]=q_n*future_price_tree[i+1][j+1]+(1-q_n)*future_price_tree[i+1][j]
    return future_price_tree

In [247]:
future_price_lattice(0.25,100,0.02,0.01,0.3,15)

array([[100.25031276,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ],
       [ 96.42578893, 104.19179181,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ],
       [ 92.74716971, 100.21690156, 108.28823553,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ],
       [ 89.20888887,  96.39365236, 104.157067  , 112.54573658,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,  

In [249]:
def future_option_price(T,S_0,r,c,sigma,n,K,m=10):
    f_p_l = future_price_lattice(T,S_0,r,c,sigma,n)
    dt = T/n
    R_n = math.exp(r*dt)
    R_C = math.exp((r-c)*dt)
    u_n = math.exp(sigma*(dt**0.5))
    d_n = 1/u_n
    q_n = (R_C-d_n)/(u_n-d_n)
    # value sequence
    f = np.vectorize(lambda x:max(x,0.0))
    payoff=f(f_p_l-K)
    stopping_time = m
    for i in range(m-1,-1,-1):
        for j in range(i+1):
            price = 1/R_n*(q_n*payoff[i+1][j+1]+(1-q_n)*payoff[i+1][j])
            if payoff[i,j]>= price:
                stopping_time = i
            else:
                payoff[i,j]= price
    return payoff[0][0],stopping_time

In [250]:
future_option_price(0.25,100,0.02,0.01,0.3,15,110.0,m=10)

(1.6626726077880374, 4)